In [1]:
import json
from pprint import pprint
import sys
import unicodedata

import fiona
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union
from django.contrib.gis.geos import GEOSGeometry
from fiona.crs import to_string, from_epsg
from fiona.transform import transform_geom
from pyproj import Proj

from madmex.models import Region, Country
from madmex.util.spatial import feature_transform

In [2]:
#name_admin = 'MEX' or column_name = 'AGUACATE_DIANA'
name_admin = 'AGUACATE_DIANA'
#column_name = 'NAME_1' or column_name = ''
column_name = ''
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/MEX_adm_shp/MEX_adm1.shp' 
#or shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/diana_poligonos_aguacate/poly_inventarios_buff30m/Inventario_1995_buff30m_poly.shp'
shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/diana_poligonos_aguacate/poly_inventarios_buff30m/Inventario_1995_buff30m_poly.shp'

In [3]:
with fiona.open(shapefile,
                encoding='utf-8') as src:
    fc = list(src)
    src_crs = src.crs
    to_string_crs = to_string(src_crs)
    proj_crs = Proj(src.crs)
    if not proj_crs.is_latlong():
        fc_proj = [feature_transform(x,"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",to_string_crs) for x in fc]
    else:
        fc_proj = fc

In [4]:
[feat['geometry']['type'] for feat in fc_proj][0:4]

['Polygon', 'Polygon', 'Polygon', 'Polygon']

In [5]:
shape_list = [shape(feat['geometry']) for feat in fc_proj]

In [6]:
country_shape = cascaded_union(shape_list)

In [7]:
country, _ = Country.objects.get_or_create(the_geom=GEOSGeometry(country_shape.wkt, 4326),
                                       name=name_admin)

In [ ]:
# Ingest features
for k in range(0,len(fc_proj)):
    shape = shape_list[k]
    geom = GEOSGeometry(shape.wkt,4326)
    
    if not column_name: #needs to generate name for every entry
        name = name_admin + '_%s'
        name_feature = name % fc_proj[k]['id']
    else:
        name_feature = fc_proj[k]['properties'][column_name].replace(" ", "_")
        name_feature_normalized = unicodedata.normalize('NFKD', name_feature).encode('ASCII', 'ignore').decode('utf-8')
        name_feature = name_admin + '_' + name_feature_normalized
    _ = Region.objects.get_or_create(name=name_feature,
                                     the_geom=geom,
                                     country=country)
    print(name_feature)

AGUACATE_DIANA_0
AGUACATE_DIANA_1
AGUACATE_DIANA_2
AGUACATE_DIANA_3
AGUACATE_DIANA_4
AGUACATE_DIANA_5
AGUACATE_DIANA_6
AGUACATE_DIANA_7
AGUACATE_DIANA_8
AGUACATE_DIANA_9
AGUACATE_DIANA_10
AGUACATE_DIANA_11
AGUACATE_DIANA_12
AGUACATE_DIANA_13
AGUACATE_DIANA_14
AGUACATE_DIANA_15
AGUACATE_DIANA_16
AGUACATE_DIANA_17
AGUACATE_DIANA_18
AGUACATE_DIANA_19
AGUACATE_DIANA_20
AGUACATE_DIANA_21
AGUACATE_DIANA_22
AGUACATE_DIANA_23
AGUACATE_DIANA_24
AGUACATE_DIANA_25
AGUACATE_DIANA_26
AGUACATE_DIANA_27
AGUACATE_DIANA_28
AGUACATE_DIANA_29
AGUACATE_DIANA_30
AGUACATE_DIANA_31
AGUACATE_DIANA_32
AGUACATE_DIANA_33
AGUACATE_DIANA_34
AGUACATE_DIANA_35
AGUACATE_DIANA_36
AGUACATE_DIANA_37
AGUACATE_DIANA_38
AGUACATE_DIANA_39
AGUACATE_DIANA_40
AGUACATE_DIANA_41
AGUACATE_DIANA_42
AGUACATE_DIANA_43
AGUACATE_DIANA_44
AGUACATE_DIANA_45
AGUACATE_DIANA_46
AGUACATE_DIANA_47
AGUACATE_DIANA_48
AGUACATE_DIANA_49
AGUACATE_DIANA_50
AGUACATE_DIANA_51
AGUACATE_DIANA_52
AGUACATE_DIANA_53
AGUACATE_DIANA_54
AGUACATE_DIANA_55
AG

In [9]:
region = 'MEX_Aguascalientes'
region_geom = Region.objects.get(name=region).the_geom
region_geom


In [10]:
region_geom.wkt

'POLYGON ((-102.126106262207 21.73960113525402, -102.133918762207 21.72841072082531, -102.1392822265624 21.72210121154797, -102.140739440918 21.71793174743652, -102.1493072509765 21.70852088928228, -102.1539459228516 21.70335006713867, -102.162757873535 21.69279289245611, -102.16886138916 21.68461036682129, -102.1762161254883 21.67797088623053, -102.1862564086914 21.67098999023432, -102.1975631713867 21.6657829284668, -102.2128067016601 21.66291046142584, -102.2231369018555 21.66045951843256, -102.2354431152344 21.65722084045416, -102.2460098266601 21.65403938293468, -102.2573165893555 21.64948081970221, -102.2666397094727 21.6452198028565, -102.2789306640625 21.64105987548828, -102.298843383789 21.63541984558117, -102.3128585815429 21.62891006469738, -102.3180313110352 21.62685966491711, -102.3313293457031 21.6233310699464, -102.3463668823242 21.6217498779298, -102.3579864501953 21.62236022949213, -102.363166809082 21.62281036376959, -102.3683624267578 21.62462043762213, -102.37754058